In [1]:
import pickle
file = 'erik_preprocessed/826bfe4bbd7594188e3d74d2539d9707b1c6a14b_45290177164699323617404875727428861719_0.cpg.pt'
# gunzip file
import gzip
import shutil

# tensor = torch.load(file)
tensor = pickle.load(open(file, 'rb'))
print(tensor)


{'edge_index': tensor([[ 0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,
          4,  5,  5,  5,  5,  6,  6,  6,  7,  7,  8,  8,  8,  8,  9,  9, 10, 10],
        [ 1,  8,  0,  4,  2,  7,  6,  1,  3,  6,  2,  6,  4,  5,  8,  1,  3,  5,
          9,  3,  4, 10,  7,  1,  2,  3,  1,  5,  0,  3,  9, 10,  4,  8,  5,  8]]), 'label': ['"AST"', '"AST"', '"AST"', '"CFG"', '"AST"', '"AST"', '"DFG"', '"AST"', '"AST"', '"AST"', '"AST"', '"CFG"', '"AST"', '"AST"', '"CG"', '"CFG"', '"AST"', '"CFG"', '"DFG"', '"AST"', '"CFG"', '"DFG"', '"DFG"', '"DFG"', '"AST"', '"CFG"', '"AST"', '"DFG"', '"AST"', '"CG"', '"AST"', '"AST"', '"DFG"', '"AST"', '"DFG"', '"AST"'], 'lexeme': ['"/mnt/efs/Erik_Dominic/diversevul/c_files/826bfe4bbd7594188e3d74d2539d9707b1c6a14b_45290177164699323617404875727428861719_0.c"', '"redraw_curbuf_later"', '""', '"redraw_buf_later"', '"curbuf"', '"type"', '""', '"type"', '"redraw_buf_later"', '"unknown0"', '"int1"'], 'enclosing': ['"redraw_curbuf_later(int type)

In [3]:
import torch
labels = ['TranslationUnitDeclaration', 'ConstructExpression','FunctionDeclaration', 'CompoundStatement', 'CallExpression', 'DeclaredReferenceExpression', 'WhileStatement', 'BinaryOperator', 'Literal', 'ReturnStatement', 'ParamVariableDeclaration', 'VariableDeclaration', 'DeclarationStatement', 'UnaryOperator', 'MemberExpression', 'ConditionalExpression', 'IfStatement', 'ForStatement', 'BreakStatement', 'MemberCallExpression', 'ArraySubscriptionExpression', 'TypeIdExpression', 'InitializerListExpression', 'ProblemDeclaration', 'CastExpression', 'ContinueStatement', 'EmptyStatement', 'GotoStatement', 'LabelStatement', 'SwitchStatement', 'CaseStatement', 'ProblemExpression', 'ExpressionList', 'DoStatement', 'DefaultStatement', 'RecordDeclaration', 'FieldDeclaration', 'ConstructorDeclaration', 'DesignatedInitializerExpression', 'EnumDeclaration', 'EnumConstantDeclaration', 'MethodDeclaration', 'ASMDeclarationStatement', 'TypedefDeclaration']
sorted_labels = sorted(labels)
label_categories = {}
# onehot encode labels
for label in sorted_labels:
    onehot = torch.zeros(len(labels))
    onehot[sorted_labels.index(label)] = 1
    label_categories[label] = onehot

In [4]:
from torch_geometric.data import InMemoryDataset, Data, HeteroData, Dataset 
from pathlib import Path 
from tqdm.auto import tqdm
import pickle
import torch
import os 

ys = []
for file in tqdm(os.listdir('erik_preprocessed')):
    if file.endswith('.pt'):
        tensor = pickle.load(open('erik_preprocessed/' + file, 'rb'))
        # y = torch.tensor([tensor['y']], dtype=torch.float32) 
        y = int(file.split('_')[-1].split('.')[0])
        ys.append(y)
        y = torch.tensor([y], dtype=torch.float32)
        
       
        edge_index = tensor['edge_index']  
        # add self_loops 
        num_nodes = tensor['codeenc'].shape[0]
        self_loops = torch.arange(num_nodes)
        self_loop_edges = torch.stack([self_loops, self_loops])
        edge_index = torch.cat([edge_index, self_loop_edges], dim=1)
        
        
        node_labels = []
        for ast in tensor['ast']:
            label= label_categories[ast.replace('"','')]
            node_labels.append(label)
        
        all_labels = torch.stack(node_labels)
        
        if num_nodes>1000 or num_nodes<10:
            continue
        x = torch.cat([tensor['codeenc'], all_labels], dim=1) 
        
        
        data = Data(x=x.to(torch.float32), edge_index=edge_index.to(torch.int64), y=y.to(torch.float32)) 
        
        # save to preprocessed
        torch.save(data, 'preprocessed/' + file)

  1%|          | 1468/245420 [00:03<09:40, 419.92it/s]


KeyboardInterrupt: 

In [5]:
x.shape

torch.Size([92, 144])

In [6]:
sum(ys)

13970

In [7]:
len(ys)

245419

In [9]:
245419/13970

17.56757337151038

In [30]:
tensor['edge_index'].dtype

torch.int64

In [24]:
tensor['ast']

['"TranslationUnitDeclaration"',
 '"FunctionDeclaration"',
 '"CompoundStatement"',
 '"CallExpression"',
 '"DeclaredReferenceExpression"',
 '"DeclaredReferenceExpression"',
 '"ReturnStatement"',
 '"ParamVariableDeclaration"',
 '"FunctionDeclaration"',
 '"ParamVariableDeclaration"',
 '"ParamVariableDeclaration"']

In [15]:
from pathlib import Path
Path(file).name

'826bfe4bbd7594188e3d74d2539d9707b1c6a14b_45290177164699323617404875727428861719_0.cpg.pt'

In [12]:
tensor['codeenc'].shape

torch.Size([11, 100])

In [ ]:

from torch_geometric.data import InMemoryDataset, Data, HeteroData
    
data = Data(x=X.cpu(), 
            edge_index=E.cpu(),
            edge_attr=None,
            y=y)